In [1]:
# ga_tune_lstm_wind.py
# pip install torch pandas scikit-learn joblib matplotlib

import argparse, math, random, time
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# ============== CONFIG (edit as you like) ==============
TIME_COL     = "TIMESTAMP"
TARGET_COL   = "TARGETVAR"
FEATURE_COLS = ["U10", "V10", "U100", "V100"]

TRAIN_RATIO  = 0.70
VAL_RATIO    = 0.15   # TEST = 1 - TRAIN - VAL

MAX_EPOCHS   = 40     # per-individual training budget
PATIENCE     = 5
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED         = 42

# GA search space
LOOKBACK_CHOICES   = [6, 12, 24, 36, 48]
H_CHOICES          = [32, 64, 128, 256]
LAYERS_CHOICES     = [1, 2, 3]
DROPOUT_CHOICES    = [0.0, 0.1, 0.2, 0.3, 0.5]
BATCH_CHOICES      = [32, 64, 128]
LR_LOG10_MIN, LR_LOG10_MAX = -4.0, -2.3   # ~1e-4 to ~5e-3
WD_CHOICES         = [0.0, 1e-5, 1e-4, 1e-3]

# GA knobs
GA_POP       = 12
GA_GENS      = 6
GA_CX_RATE   = 0.6   # crossover prob
GA_MUT_RATE  = 0.25  # mutation prob
ELITE_K      = 2     # carry-best each gen

# ============== Utilities ==============
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def smape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return 100.0 * np.mean(2.0 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true) + eps))

class SeqDS(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.fc = nn.Linear(hidden_size, 1)
    def forward(self, x):
        out, _ = self.lstm(x)      # (B, T, H)
        return self.fc(out[:, -1, :])

def make_sequences(X, y, lookback):
    X_seq, y_seq = [], []
    for i in range(lookback, len(X)):
        X_seq.append(X[i-lookback:i, :])
        y_seq.append(y[i, 0])
    X_seq = np.array(X_seq, dtype=np.float32)
    y_seq = np.array(y_seq, dtype=np.float32).reshape(-1, 1)
    return X_seq, y_seq

# ============== Data prep (once) ==============
def load_and_split(file_path, sheet):
    df = pd.read_excel(file_path, sheet_name=sheet)
    df[TIME_COL] = pd.to_datetime(df[TIME_COL], infer_datetime_format=True, errors="coerce")
    df = df.sort_values(TIME_COL).reset_index(drop=True)
    data = df[[TIME_COL, TARGET_COL] + FEATURE_COLS].dropna().reset_index(drop=True)

    n = len(data)
    n_train = int(n * TRAIN_RATIO)
    n_val   = int(n * (TRAIN_RATIO + VAL_RATIO))

    train_df = data.iloc[:n_train].reset_index(drop=True)
    val_df   = data.iloc[n_train:n_val].reset_index(drop=True)
    test_df  = data.iloc[n_val:].reset_index(drop=True)

    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    X_train = x_scaler.fit_transform(train_df[FEATURE_COLS].values.astype(np.float32))
    y_train = y_scaler.fit_transform(train_df[[TARGET_COL]].values.astype(np.float32))
    X_val   = x_scaler.transform(val_df[FEATURE_COLS].values.astype(np.float32))
    y_val   = y_scaler.transform(val_df[[TARGET_COL]].values.astype(np.float32))
    X_test  = x_scaler.transform(test_df[FEATURE_COLS].values.astype(np.float32))
    y_test  = y_scaler.transform(test_df[[TARGET_COL]].values.astype(np.float32))

    return (train_df, val_df, test_df,
            X_train, y_train, X_val, y_val, X_test, y_test,
            x_scaler, y_scaler)

# ============== Training for a single individual ==============
def train_once(Xtr, ytr, Xv, yv, y_scaler, params, max_epochs=MAX_EPOCHS, patience=PATIENCE):
    lookback   = params["lookback"]
    hidden     = params["hidden"]
    layers     = params["layers"]
    dropout    = params["dropout"]
    batch_size = params["batch"]
    lr         = params["lr"]
    wd         = params["weight_decay"]

    # sequences for this lookback
    Xtr_seq, ytr_seq = make_sequences(Xtr, ytr, lookback)
    Xv_seq,  yv_seq  = make_sequences(Xv,  yv,  lookback)

    # guard (shouldn't happen with our dataset, but be safe)
    if len(Xtr_seq) < 10 or len(Xv_seq) < 10:
        return float("inf"), None

    train_loader = DataLoader(SeqDS(Xtr_seq, ytr_seq), batch_size=batch_size, shuffle=True, drop_last=False)
    val_loader   = DataLoader(SeqDS(Xv_seq,  yv_seq ), batch_size=batch_size, shuffle=False, drop_last=False)

    model = LSTMRegressor(len(FEATURE_COLS), hidden, layers, dropout).to(DEVICE)
    criterion = nn.MSELoss()
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    best_val = float("inf")
    bad = 0
    best_state = None

    for epoch in range(1, max_epochs+1):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            opt.step()

        # val
        model.eval()
        losses = []
        y_true_s, y_pred_s = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(DEVICE)
                preds = model(xb).cpu().numpy()
                y_pred_s.append(preds)
                y_true_s.append(yb.numpy())
                losses.append(criterion(model(xb), yb.to(DEVICE)).item())

        # compute RMSE in original units for selection
        y_pred_s = np.vstack(y_pred_s); y_true_s = np.vstack(y_true_s)
        y_pred = y_scaler.inverse_transform(y_pred_s).ravel()
        y_true = y_scaler.inverse_transform(y_true_s).ravel()
        rmse = math.sqrt(mean_squared_error(y_true, y_pred))

        if rmse < best_val - 1e-6:
            best_val = rmse
            bad = 0
            best_state = { "model": model.state_dict(), "params": params.copy() }
        else:
            bad += 1
            if bad >= patience:
                break

    return best_val, best_state

# ============== GA bits ==============
def random_individual():
    # genome layout: [lookback_idx, hidden_idx, layers_idx, dropout_idx, batch_idx, lr_log10, wd_idx]
    return [
        random.randrange(len(LOOKBACK_CHOICES)),
        random.randrange(len(H_CHOICES)),
        random.randrange(len(LAYERS_CHOICES)),
        random.randrange(len(DROPOUT_CHOICES)),
        random.randrange(len(BATCH_CHOICES)),
        random.uniform(LR_LOG10_MIN, LR_LOG10_MAX),
        random.randrange(len(WD_CHOICES)),
    ]

def decode(ind):
    return {
        "lookback": LOOKBACK_CHOICES[ind[0]],
        "hidden":   H_CHOICES[ind[1]],
        "layers":   LAYERS_CHOICES[ind[2]],
        "dropout":  DROPOUT_CHOICES[ind[3]],
        "batch":    BATCH_CHOICES[ind[4]],
        "lr":       10.0 ** ind[5],
        "weight_decay": WD_CHOICES[ind[6]],
    }

def crossover(p1, p2):
    if random.random() > GA_CX_RATE:
        return p1[:], p2[:]
    cut = random.randint(1, len(p1)-1)
    c1 = p1[:cut] + p2[cut:]
    c2 = p2[:cut] + p1[cut:]
    return c1, c2

def mutate(ind):
    if random.random() < GA_MUT_RATE:
        g = random.randrange(len(ind))
        if g in [0,1,2,3,4,6]:  # discrete genes
            if g == 0: ind[0] = random.randrange(len(LOOKBACK_CHOICES))
            if g == 1: ind[1] = random.randrange(len(H_CHOICES))
            if g == 2: ind[2] = random.randrange(len(LAYERS_CHOICES))
            if g == 3: ind[3] = random.randrange(len(DROPOUT_CHOICES))
            if g == 4: ind[4] = random.randrange(len(BATCH_CHOICES))
            if g == 6: ind[6] = random.randrange(len(WD_CHOICES))
        else:  # lr_log10 (continuous) -> small Gaussian step
            ind[5] += random.gauss(0, 0.2)
            ind[5] = max(min(ind[5], LR_LOG10_MAX), LR_LOG10_MIN)
    return ind

def tournament_select(pop, k=3):
    # pop elements are tuples (fitness, ind, state)
    cands = random.sample(pop, k)
    cands.sort(key=lambda x: x[0])  # lower fitness (RMSE) is better
    return cands[0][1][:]  # return a copy of genome

# ============== Main ==============
def run_ga(file_path, sheet):
    set_seed(SEED)
    # data (scaled once)
    (train_df, val_df, test_df,
     Xtr, ytr, Xv, yv, Xte, yte,
     x_scaler, y_scaler) = load_and_split(file_path, sheet)

    print(f"Data span: {train_df[TIME_COL].min()} → {test_df[TIME_COL].max()}")
    print(f"Splits (rows): train={len(train_df)} val={len(val_df)} test={len(test_df)}")

    # init population
    pop = []
    for i in range(GA_POP):
        ind = random_individual()
        params = decode(ind)
        fit, state = train_once(Xtr, ytr, Xv, yv, y_scaler, params)
        pop.append((fit, ind, state))
        print(f"Init {i+1}/{GA_POP} -> RMSE={fit:.5f}, {params}")

    # evolve
    best = min(pop, key=lambda x: x[0])
    print(f"\nGen 0 best: RMSE={best[0]:.5f}, params={decode(best[1])}")

    for gen in range(1, GA_GENS+1):
        new_pop = sorted(pop, key=lambda x: x[0])[:ELITE_K]  # elitism

        while len(new_pop) < GA_POP:
            p1 = tournament_select(pop); p2 = tournament_select(pop)
            c1, c2 = crossover(p1, p2)
            c1 = mutate(c1); c2 = mutate(c2)

            for child in [c1, c2]:
                params = decode(child)
                fit, state = train_once(Xtr, ytr, Xv, yv, y_scaler, params)
                new_pop.append((fit, child, state))
                print(f"[Gen {gen}] cand -> RMSE={fit:.5f}, {params}")
                if len(new_pop) >= GA_POP: break

        pop = new_pop
        best = min(pop, key=lambda x: x[0])
        print(f"Gen {gen} best: RMSE={best[0]:.5f}, params={decode(best[1])}")

    # best individual
    best_fit, best_ind, best_state = min(pop, key=lambda x: x[0])
    best_params = decode(best_ind)
    print("\n==== GA RESULT ====")
    print(f"Best Val RMSE: {best_fit:.6f}")
    print("Best Params:", best_params)

    # Retrain best on TRAIN+VAL, evaluate on TEST, save artifacts
    X_trval = np.vstack([Xtr, Xv])
    y_trval = np.vstack([ytr, yv])
    lookback = best_params["lookback"]
    Xtrv_seq, ytrv_seq = make_sequences(X_trval, y_trval, lookback)
    Xte_seq, yte_seq   = make_sequences(Xte,    yte,    lookback)

    train_loader = DataLoader(SeqDS(Xtrv_seq, ytrv_seq), batch_size=best_params["batch"], shuffle=True)
    test_loader  = DataLoader(SeqDS(Xte_seq,  yte_seq ), batch_size=best_params["batch"], shuffle=False)

    model = LSTMRegressor(len(FEATURE_COLS), best_params["hidden"], best_params["layers"], best_params["dropout"]).to(DEVICE)
    if best_state and "model" in best_state and best_state["params"] == best_params:
        # warm-start from the best state found on (Train/Val) if hyperparams identical
        model.load_state_dict(best_state["model"])

    crit = nn.MSELoss()
    opt  = torch.optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])

    best_val = float("inf")
    bad = 0
    # quick early stopping on a small split out of train+val to avoid overfitting
    # (alternatively, you can just run fixed epochs since we're on final fit)
    for epoch in range(1, MAX_EPOCHS+1):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad(); loss = crit(model(xb), yb); loss.backward(); opt.step()

    # Evaluate on TEST (original units)
    model.eval()
    y_true_s, y_pred_s = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            preds = model(xb.to(DEVICE)).cpu().numpy()
            y_pred_s.append(preds)
            y_true_s.append(yb.numpy())
    y_pred_s = np.vstack(y_pred_s); y_true_s = np.vstack(y_true_s)
    y_pred = y_scaler.inverse_transform(y_pred_s).ravel()
    y_true = y_scaler.inverse_transform(y_true_s).ravel()

    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    smp  = smape(y_true, y_pred)

    outdir = Path(".")
    torch.save(model.state_dict(), outdir / "lstm_wind_best_ga.pt")
    joblib.dump(x_scaler, outdir / "feature_scaler_ga.pkl")
    joblib.dump(y_scaler, outdir / "target_scaler_ga.pkl")

    print("\n==== TEST METRICS (retrained on Train+Val) ====")
    print(f"RMSE : {rmse:.6f}")
    print(f"MAE  : {mae:.6f}")
    print(f"R^2  : {r2:.6f}")
    print(f"sMAPE: {smp:.2f}%")
    print("Saved: lstm_wind_best_ga.pt, feature_scaler_ga.pkl, target_scaler_ga.pkl")

    return best_params

def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--file",  type=str, default="WindPowerForecastingData.xlsx")
    p.add_argument("--sheet", type=str, default="WindPowerForecastingData")
    return p.parse_args()

if __name__ == "__main__":
    set_seed()
#args = parse_args()
    a = "WindPowerForecastingData.xlsx"
    b = "WindPowerForecastingData"
    run_ga(a, b)


/tmp/ipykernel_1556/2842457548.py:90: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[TIME_COL] = pd.to_datetime(df[TIME_COL], infer_datetime_format=True, errors="coerce")


Data span: 2012-01-01 01:00:00 → 2013-11-30 00:00:00
Splits (rows): train=11735 val=2515 test=2515
Init 1/12 -> RMSE=0.19571, {'lookback': 6, 'hidden': 32, 'layers': 3, 'dropout': 0.2, 'batch': 32, 'lr': 0.0002395842446100862, 'weight_decay': 0.0}
Init 2/12 -> RMSE=0.20622, {'lookback': 48, 'hidden': 32, 'layers': 3, 'dropout': 0.3, 'batch': 32, 'lr': 0.00011237126581726528, 'weight_decay': 1e-05}
Init 3/12 -> RMSE=0.24445, {'lookback': 12, 'hidden': 32, 'layers': 3, 'dropout': 0.1, 'batch': 128, 'lr': 0.0012729271343568762, 'weight_decay': 0.001}
Init 4/12 -> RMSE=0.18832, {'lookback': 12, 'hidden': 256, 'layers': 3, 'dropout': 0.2, 'batch': 32, 'lr': 0.0019497212456684896, 'weight_decay': 1e-05}
Init 5/12 -> RMSE=0.18871, {'lookback': 36, 'hidden': 128, 'layers': 2, 'dropout': 0.1, 'batch': 32, 'lr': 0.004238988347579313, 'weight_decay': 0.0001}
Init 6/12 -> RMSE=0.18538, {'lookback': 6, 'hidden': 32, 'layers': 2, 'dropout': 0.0, 'batch': 64, 'lr': 0.0027589283556667516, 'weight_deca